# Using OpenAI with ROS

<img src="img/robotignite_logo_text.png" width="700" />

## Unit 3: How to create a new Robot Environment for a new Robot

<p style="background:green;color:white;">SUMMARY</p>

Estimated time to completion: <b>2 hours</b>

In this unit, you will learn how to create a Robot Environment for a Moving Cube with a single disk in the roll axis using the **OpenAI ROS** structure. We will call it **MyCubeSingleDiskEnv**.

<p style="background:green;color:white;">END OF SUMMARY</p>

# Use the power of OpenAI through simple Robot-Gazebo-Env

So, you now understand how the **CartPole Environments structure** works. Now, we need to create one from scratch for the One-Disk-Moving-Cube. There are three main steps:

* We need to create a **Robot Environment** that has the basic functions needed to use the RoboCube.
* We need to decide how we move the RoboCube and how to get the sensor data.
* We need to create functions that allow environments that inherit from our class to retrieve sensor data and access the cube's functionality, without knowing all the ROS related stuff.

With this created, you will have a **Robot Environment** that can be used by any **Task Environment** that you create afterwards.

###  0. Create a package to place all your code in

We will first create a package that will hold all the code that you generate in these next chapters.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd; cd ../src; 

In [ ]:
catkin_create_pkg my_moving_cube_pkg rospy openai_ros

In [ ]:
cd my_moving_cube_pkg/

In [ ]:
mkdir scripts; cd scripts

In [ ]:
touch my_cube_single_disk_env.py;chmod +x my_cube_single_disk_env.py

###  1. Create a basic Robot Environment

We will start from an empty template that we give you here, which has all the functions needed for the **RobotGazeboEnv**.

<p style="background:#407EAF;color:white;">**template_my_robot_env.py**</p>

In [ ]:
from openai_ros import robot_gazebo_env


class MyRobotEnv(robot_gazebo_env.RobotGazeboEnv):
    """Superclass for all Robot environments.
    """

    def __init__(self):
        """Initializes a new Robot environment.
        """
        # Variables that we give through the constructor.

        # Internal Vars
        self.controllers_list = ['my_robot_controller1','my_robot_controller2', ..., 'my_robot_controllerX']

        self.robot_name_space = "my_robot_namespace"

        reset_controls_bool = True or False
        
        # We launch the init function of the Parent Class robot_gazebo_env.RobotGazeboEnv
        
        super(MyRobotEnv, self).__init__(controllers_list=self.controllers_list,
                                                robot_name_space=self.robot_name_space,
                                                reset_controls=reset_controls_bool)

    # Methods needed by the RobotGazeboEnv
    # ----------------------------
    
    

    def _check_all_systems_ready(self):
        """
        Checks that all the sensors, publishers and other simulation systems are
        operational.
        """
        # TODO
        return True
    
    # Methods that the TaskEnvironment will need to define here as virtual
    # because they will be used in RobotGazeboEnv GrandParentClass and defined in the
    # TaskEnvironment.
    # ----------------------------
    def _set_init_pose(self):
        """Sets the Robot in its init pose
        """
        raise NotImplementedError()
    
    
    def _init_env_variables(self):
        """Inits variables needed to be initialised each time we reset at the start
        of an episode.
        """
        raise NotImplementedError()

    def _compute_reward(self, observations, done):
        """Calculates the reward to give based on the observations given.
        """
        raise NotImplementedError()

    def _set_action(self, action):
        """Applies the given action to the simulation.
        """
        raise NotImplementedError()

    def _get_obs(self):
        raise NotImplementedError()

    def _is_done(self, observations):
        """Checks if episode done based on observations given.
        """
        raise NotImplementedError()
        
    # Methods that the TaskEnvironment will need.
    # ----------------------------

You can see here that the template is divided into four parts:

* 1.1- Initialization of the class **MyRobotEnv**, or whatever name you choose.
* 1.2- Definition of the methods needed by the Gazebo Environment, which were declared virtually inside RobotGazeboEnv.
* 1.3- Virtual definition of methods that the Task Environment will need to define here as virtual because they will be used in the RobotGazeboEnv GrandParentClass and defined in the Task Environment.
* 1.4- Definition of methods that the Task Environment will need.

#### 1.1- Initialization of the class MyRobotEnv

In [ ]:
from openai_ros import robot_gazebo_env

class MyRobotEnv(robot_gazebo_env.RobotGazeboEnv):
    """Superclass for all CubeSingleDisk environments.
    """

    def __init__(self, init_roll_vel):
        """Initializes a new CubeSingleDisk environment.
        """
        # Variables that we give through the constructor.
        self.init_roll_vel = init_roll_vel

        self.controllers_list = ['my_robot_controller1','my_robot_controller2', ..., 'my_robot_controllerX']

        self.robot_name_space = "my_robot_namespace"

        reset_controls_bool = True or False
        
        # We launch the init function of the Parent Class robot_gazebo_env.RobotGazeboEnv
        
        super(CubeSingleDiskEnv, self).__init__(controllers_list=self.controllers_list,
                                                robot_name_space=self.robot_name_space,
                                                reset_controls=reset_controls_bool)

Here we import the **robot_gazebo_env** from the python module folder openai_ros. Inside **robot_gazebo_env**, we find the class **RobotGazeboEnv**. In the class definition of **MyRobotEnv**, we give inheritance to **robot_gazebo_env.RobotGazeboEnv**. That means to look for the python module file **robot_gazebo_env**, and inside, get the class **RobotGazeboEnv**.

In [ ]:
from openai_ros import robot_gazebo_env

class MyRobotEnv(robot_gazebo_env.RobotGazeboEnv):

Then, you have to change the **MyRobotEnv** to the class name you prefer; in this case, let's call it **MyCubeSingleDiskEnv**.

In [ ]:
class MyCubeSingleDiskEnv(robot_gazebo_env.RobotGazeboEnv):

In [ ]:
def __init__(self, init_roll_vel):
    # Variables that we give through the constructor.
    self.init_roll_vel = init_roll_vel

In this case, as you can see, we added a variable in the init function. These variables are the ones you want the **Task Environment** to pass to this new Robot Environment. We need it to pass the speed at the start of each episode to set the roll disk. For this scenario, it will most likely always be **0.0**, but it could change depending on the **Task Environment**.

Now we define some variables that need to be passed to the **RobotGazeboEnv** super constructor method __init__. These are: **controllers_list, robot_name_space, and reset_controls**. These variables are used by the RobotGazeboEnv to know which controllers to reset each time a learning episode starts.

We can make the controllers available in the simulation by just executing the following commands in the web shell:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosservice call /moving_cube/controller_manager/list_controllers "{}"

<table class="transparent float_l">
<tr>
<th>
<p style="background: #407EAF">WebShell #1 Output</p>
</th>
</tr>
</table>

In [ ]:
controller:
  -
    name: "joint_state_controller"
    state: "running"
    type: "joint_state_controller/JointStateController"
    claimed_resources:
      -
        hardware_interface: "hardware_interface::JointStateInterface"
        resources: []
  -
    name: "inertia_wheel_roll_joint_velocity_controller"
    state: "running"
    type: "effort_controllers/JointVelocityController"
    claimed_resources:
      -
        hardware_interface: "hardware_interface::EffortJointInterface"
        resources: [inertia_wheel_roll_joint]

So, the list of controllers should look something like this:

In [ ]:
self.controllers_list = ['joint_state_controller','inertia_wheel_roll_joint_velocity_controller']

And to get the namespace used in the controllers, just execute the following command to see all the controllers' namespace: 

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

<table class="transparent float_l">
<tr>
<th>
<p style="background: #407EAF">WebShell #1 Output</p>
</th>
</tr>
</table>

So. you should have the following variable:

In [ ]:
self.robot_name_space = "moving_cube"

Here we decide if we want to reset the controllers or not:

In [ ]:
reset_controls_bool = True

And finally, you pass it to the creation **__init__** function:

In [ ]:
super(MyCubeSingleDiskEnv, self).__init__(controllers_list=self.controllers_list,
                                                robot_name_space=self.robot_name_space,
                                                reset_controls=reset_controls_bool)

#### 1.2- Definition of the Methods needed by the RobotGazeboEnv

These are methods that RobotGazeboEnv needs to have defined somewhere. And the place to be defined is in this RobotEnvironment because it has access to the robot.

In [ ]:
def _check_all_systems_ready(self):
    """
    Checks that all the sensors, publishers and other simulation systems are
    operational.
    """
    # TODO
    return True

We have to return to the __init__ method and create all the subscribers and publishers for the One Disk Cube so that the check_all_systems_ready can work.

In [1]:
    def __init__(self):
        """Initializes a new CubeSingleDisk environment.

        Args:
        """
        # Variables that we give through the constructor.
        # None

        
        self.controllers_list = ['joint_state_controller',
                                 'inertia_wheel_roll_joint_velocity_controller'
                                 ]

        self.robot_name_space = "moving_cube"

        # We launch the init function of the Parent Class robot_gazebo_env.RobotGazeboEnv
        super(MyCubeSingleDiskEnv, self).__init__(controllers_list=self.controllers_list,
                                                robot_name_space=self.robot_name_space,
                                                reset_controls=True)



        """
        To check any topic we need to have the simulations running, we need to do two things:
        1) Unpause the simulation: without that the stream of data doesn't flow. This is for simulations
        that are pause for whatever reason
        2) If the simulation was running already for some reason, we need to reset the controllers.
        This has to do with the fact that some plugins with tf don't understand the reset of the simulation
        and need to be reset to work properly.
        """
        self.gazebo.unpauseSim()
        self.controllers_object.reset_controllers()
        self._check_all_sensors_ready()

        # We Start all the ROS related Subscribers and publishers
        rospy.Subscriber("/moving_cube/joint_states", JointState, self._joints_callback)
        rospy.Subscriber("/moving_cube/odom", Odometry, self._odom_callback)

        self._roll_vel_pub = rospy.Publisher('/moving_cube/inertia_wheel_roll_joint_velocity_controller/command',
                                             Float64, queue_size=1)

        self._check_publishers_connection()

        self.gazebo.pauseSim()

Here we create the **Subscribers** and **Publishers**. The important part is the **unpause**, **check_all_sensors_ready**, and **pause**. These are key to being able to reset the controllers and read the sensors. We use the objects created in the RobotGazeboEnv parent class so that we have access to it without having to know how it works.

In [ ]:
self.gazebo.unpauseSim()
self.controllers_object.reset_controllers()

Here we unpause the simulation. This allows us to reset controllers and make the first test to see if the sensors are working. Checking this is key for Ai learning because we need a reliable sensor and controller communication.

In [ ]:
self._check_all_sensors_ready()

**Note** that inside this **MyCubeSingleDiskEnv** we use **_check_all_sensors_ready** which is an internal function, while the **RobotGazeboEnv** parent class will call the **_check_all_systems_ready**. We could also just useone function, but its separeted here to show the diference in who uses what function.

We have to define the method inside this class, like so:

In [ ]:
    def _check_all_systems_ready(self):
        """
        Checks that all the sensors, publishers and other simulation systems are
        operational.
        """
        self._check_all_sensors_ready()
        return True


    # CubeSingleDiskEnv virtual methods
    # ----------------------------

    def _check_all_sensors_ready(self):
        self._check_joint_states_ready()
        self._check_odom_ready()
        rospy.logdebug("ALL SENSORS READY")

    def _check_joint_states_ready(self):
        self.joints = None
        while self.joints is None and not rospy.is_shutdown():
            try:
                self.joints = rospy.wait_for_message("/moving_cube/joint_states", JointState, timeout=1.0)
                rospy.logdebug("Current moving_cube/joint_states READY=>" + str(self.joints))

            except:
                rospy.logerr("Current moving_cube/joint_states not ready yet, retrying for getting joint_states")
        return self.joints

    def _check_odom_ready(self):
        self.odom = None
        while self.odom is None and not rospy.is_shutdown():
            try:
                self.odom = rospy.wait_for_message("/moving_cube/odom", Odometry, timeout=1.0)
                rospy.logdebug("Current /moving_cube/odom READY=>" + str(self.odom))

            except:
                rospy.logerr("Current /moving_cube/odom not ready yet, retrying for getting odom")

        return self.odom

In the case of the OneDiskCube, for sensors, we only have the odometry that tells us where the Cube Body is in the simulated world (**/moving_cube/odom**), and how the disk joint (speed, position, efforts) is through the **/moving_cube/joint_states**. 

Once we have this, we know that ROS can establish a connection to these topics and they're ready, so we can declare the susbcribers now. So, let's go back to the __init__ method:

In [ ]:
rospy.Subscriber("/moving_cube/joint_states", JointState, self._joints_callback)
rospy.Subscriber("/moving_cube/odom", Odometry, self._odom_callback)

We add the necessary imports:

In [ ]:
import rospy
from sensor_msgs.msg import JointState
from nav_msgs.msg import Odometry

And we also have to declare their callbacks, which will start to store the sensor data in **self.joints** and **self.odom**. Now we will always have the most updated sensor data, even when we pause the simulation, having updated observations for the learning algorithms.

In [ ]:
    def _joints_callback(self, data):
        self.joints = data
    
    def _odom_callback(self, data):
        self.odom = data

We add the publisher now:

In [ ]:
self._roll_vel_pub = rospy.Publisher('/moving_cube/inertia_wheel_roll_joint_velocity_controller/command',
                                             Float64, queue_size=1)

And the necessary imports:

In [ ]:
from std_msgs.msg import Float64

And now we define the **_check_publishers_connection**, to check that our publisher is ready to receive the speed commands and doesn't lose any messages:

In [ ]:
    def _check_publishers_connection(self):
        """
        Checks that all the publishers are working
        :return:
        """
        rate = rospy.Rate(10)  # 10hz
        while self._roll_vel_pub.get_num_connections() == 0 and not rospy.is_shutdown():
            rospy.logdebug("No susbribers to _roll_vel_pub yet so we wait and try again")
            try:
                rate.sleep()
            except rospy.ROSInterruptException:
                # This is to avoid error when world is rested, time when backwards.
                pass
        rospy.logdebug("_roll_vel_pub Publisher Connected")

        rospy.logdebug("All Publishers READY")

And now we have to define the method that will move the disks, publishing in our **self._roll_vel_pub** publisher, called **move_joints**. This method will be used internally in **MyCubeSingleDiskEnv** and also in the **Task Environment**. We go to this section of the template:

In [ ]:
    # Methods that the TaskEnvironment will need.
    # ----------------------------

We place our code here because it's a function that will be used by the **TaskEnv**, and this way, we keep the code organized:

In [ ]:
    def move_joints(self, roll_speed):
        joint_speed_value = Float64()
        joint_speed_value.data = roll_speed
        rospy.logdebug("Single Disk Roll Velocity>>" + str(joint_speed_value))
        self._roll_vel_pub.publish(joint_speed_value)
        self.wait_until_roll_is_in_vel(joint_speed_value.data)
    
    def wait_until_roll_is_in_vel(self, velocity):
    
        rate = rospy.Rate(10)
        start_wait_time = rospy.get_rostime().to_sec()
        end_wait_time = 0.0
        epsilon = 0.1
        v_plus = velocity + epsilon
        v_minus = velocity - epsilon
        while not rospy.is_shutdown():
            joint_data = self._check_joint_states_ready()
            roll_vel = joint_data.velocity[0]
            rospy.logdebug("VEL=" + str(roll_vel) + ", ?RANGE=[" + str(v_minus) + ","+str(v_plus)+"]")
            are_close = (roll_vel <= v_plus) and (roll_vel > v_minus)
            if are_close:
                rospy.logdebug("Reached Velocity!")
                end_wait_time = rospy.get_rostime().to_sec()
                break
            rospy.logdebug("Not there yet, keep waiting...")
            rate.sleep()
        delta_time = end_wait_time- start_wait_time
        rospy.logdebug("[Wait Time=" + str(delta_time)+"]")
        return delta_time

It essentially gets a given roll speed and publishes that speed through the ROS publisher **self._roll_vel_pub**. The last part is also vital because it guarantees that all the actions are executed and aren't overrun by the next one. This is the method **wait_until_roll_is_in_vel**. This method will wait until the roll disk wheel reaches the desired speed, with a certain error. 

#### 1.3- Virtual definition of methods that the Task Environment will need to define

These methods are basically virtual methods that allow them to be used in **RobotGazeboEnv** and **MyCubeSingleDiskEnv**, but have to be defined upstream in the **TaskEnvironment**. We do this because these methods relate to how the Task is to be learned. So, it has to be defined on that level.

In [2]:
    # Methods that the TaskEnvironment will need to define here as virtual
    # because they will be used in RobotGazeboEnv GrandParentClass and defined in the
    # TaskEnvironment.
    # ----------------------------
    def _set_init_pose(self):
        """Sets the Robot in its init pose
        """
        raise NotImplementedError()
    
    def _init_env_variables(self):
        """Inits variables needed to be initialised each time we reset at the start
        of an episode.
        """
        raise NotImplementedError()

    def _compute_reward(self, observations, done):
        """Calculates the reward to give based on the observations given.
        """
        raise NotImplementedError()

    def _set_action(self, action):
        """Applies the given action to the simulation.
        """
        raise NotImplementedError()

    def _get_obs(self):
        raise NotImplementedError()

    def _is_done(self, observations):
        """Checks if episode done based on observations given.
        """
        raise NotImplementedError()

#### 1.4- Definition of Methods that the TaskEnvironment will need

These methods will be used by the **TaskEnvironment**, but need to be defined in this RobotEvironment in order to have the necessary access to publishers and subscribers.

In [ ]:
def move_joints(self, roll_speed):
    joint_speed_value = Float64()
    joint_speed_value.data = roll_speed
    rospy.logdebug("Single Disk Roll Velocity>>" + str(joint_speed_value))
    self._roll_vel_pub.publish(joint_speed_value)
    self.wait_until_roll_is_in_vel(joint_speed_value.data)

def wait_until_roll_is_in_vel(self, velocity):

    rate = rospy.Rate(10)
    start_wait_time = rospy.get_rostime().to_sec()
    end_wait_time = 0.0
    epsilon = 0.1
    v_plus = velocity + epsilon
    v_minus = velocity - epsilon
    while not rospy.is_shutdown():
        joint_data = self._check_joint_states_ready()
        roll_vel = joint_data.velocity[0]
        rospy.logdebug("VEL=" + str(roll_vel) + ", ?RANGE=[" + str(v_minus) + ","+str(v_plus)+"]")
        are_close = (roll_vel <= v_plus) and (roll_vel > v_minus)
        if are_close:
            rospy.logdebug("Reached Velocity!")
            end_wait_time = rospy.get_rostime().to_sec()
            break
        rospy.logdebug("Not there yet, keep waiting...")
        rate.sleep()
    delta_time = end_wait_time- start_wait_time
    rospy.logdebug("[Wait Time=" + str(delta_time)+"]")
    return delta_time

def get_joints(self):
    return self.joints

def get_odom(self):
    return self.odom

We define only RAW data methods to access the sensor data **get_joints** and **get_odom**. The **move_joints** are also used by the **TaskEnv**, but we defined them already in the **_set_init_pose** method.

And don't forget the imports:

In [1]:
import numpy

## Final Result

<p style="background:#407EAF;color:white;">**my_cube_single_disk_env.py**</p>

In [ ]:
import numpy
import rospy
from openai_ros import robot_gazebo_env
from std_msgs.msg import Float64
from sensor_msgs.msg import JointState
from nav_msgs.msg import Odometry



class MyCubeSingleDiskEnv(robot_gazebo_env.RobotGazeboEnv):
    """Superclass for all CubeSingleDisk environments.
    """

    def __init__(self):
        """Initializes a new CubeSingleDisk environment.

        Args:
        """
        # Variables that we give through the constructor.
        # None in this case

        # Internal Vars
        self.controllers_list = ['joint_state_controller',
                                 'inertia_wheel_roll_joint_velocity_controller'
                                 ]

        self.robot_name_space = "moving_cube"

        # We launch the init function of the Parent Class robot_gazebo_env.RobotGazeboEnv
        super(MyCubeSingleDiskEnv, self).__init__(controllers_list=self.controllers_list,
                                                robot_name_space=self.robot_name_space,
                                                reset_controls=True)



        """
        To check any topic we need to have the simulations running, we need to do two things:
        1) Unpause the simulation: without that th stream of data doesnt flow. This is for simulations
        that are pause for whatever the reason
        2) If the simulation was running already for some reason, we need to reset the controlers.
        This has to do with the fact that some plugins with tf, dont understand the reset of the simulation
        and need to be reseted to work properly.
        """
        self.gazebo.unpauseSim()
        self.controllers_object.reset_controllers()
        self._check_all_sensors_ready()

        # We Start all the ROS related Subscribers and publishers
        rospy.Subscriber("/moving_cube/joint_states", JointState, self._joints_callback)
        rospy.Subscriber("/moving_cube/odom", Odometry, self._odom_callback)

        self._roll_vel_pub = rospy.Publisher('/moving_cube/inertia_wheel_roll_joint_velocity_controller/command',
                                             Float64, queue_size=1)

        self._check_publishers_connection()

        self.gazebo.pauseSim()

    # Methods needed by the RobotGazeboEnv
    # ----------------------------
    

    def _check_all_systems_ready(self):
        """
        Checks that all the sensors, publishers and other simulation systems are
        operational.
        """
        self._check_all_sensors_ready()
        return True


    # CubeSingleDiskEnv virtual methods
    # ----------------------------

    def _check_all_sensors_ready(self):
        self._check_joint_states_ready()
        self._check_odom_ready()
        rospy.logdebug("ALL SENSORS READY")

    def _check_joint_states_ready(self):
        self.joints = None
        while self.joints is None and not rospy.is_shutdown():
            try:
                self.joints = rospy.wait_for_message("/moving_cube/joint_states", JointState, timeout=1.0)
                rospy.logdebug("Current moving_cube/joint_states READY=>" + str(self.joints))

            except:
                rospy.logerr("Current moving_cube/joint_states not ready yet, retrying for getting joint_states")
        return self.joints

    def _check_odom_ready(self):
        self.odom = None
        while self.odom is None and not rospy.is_shutdown():
            try:
                self.odom = rospy.wait_for_message("/moving_cube/odom", Odometry, timeout=1.0)
                rospy.logdebug("Current /moving_cube/odom READY=>" + str(self.odom))

            except:
                rospy.logerr("Current /moving_cube/odom not ready yet, retrying for getting odom")

        return self.odom
        
    def _joints_callback(self, data):
        self.joints = data
    
    def _odom_callback(self, data):
        self.odom = data
        
    def _check_publishers_connection(self):
        """
        Checks that all the publishers are working
        :return:
        """
        rate = rospy.Rate(10)  # 10hz
        while self._roll_vel_pub.get_num_connections() == 0 and not rospy.is_shutdown():
            rospy.logdebug("No susbribers to _roll_vel_pub yet so we wait and try again")
            try:
                rate.sleep()
            except rospy.ROSInterruptException:
                # This is to avoid error when world is rested, time when backwards.
                pass
        rospy.logdebug("_roll_vel_pub Publisher Connected")

        rospy.logdebug("All Publishers READY")
    
    # Methods that the TaskEnvironment will need to define here as virtual
    # because they will be used in RobotGazeboEnv GrandParentClass and defined in the
    # TaskEnvironment.
    # ----------------------------
    def _set_init_pose(self):
        """Sets the Robot in its init pose
        """
        raise NotImplementedError()
    
    def _init_env_variables(self):
        """Inits variables needed to be initialised each time we reset at the start
        of an episode.
        """
        raise NotImplementedError()

    def _compute_reward(self, observations, done):
        """Calculates the reward to give based on the observations given.
        """
        raise NotImplementedError()

    def _set_action(self, action):
        """Applies the given action to the simulation.
        """
        raise NotImplementedError()

    def _get_obs(self):
        raise NotImplementedError()

    def _is_done(self, observations):
        """Checks if episode done based on observations given.
        """
        raise NotImplementedError()
        
    # Methods that the TaskEnvironment will need.
    # ----------------------------
    def move_joints(self, roll_speed):
        joint_speed_value = Float64()
        joint_speed_value.data = roll_speed
        rospy.logdebug("Single Disk Roll Velocity>>" + str(joint_speed_value))
        self._roll_vel_pub.publish(joint_speed_value)
        self.wait_until_roll_is_in_vel(joint_speed_value.data)
    
    def wait_until_roll_is_in_vel(self, velocity):
    
        rate = rospy.Rate(10)
        start_wait_time = rospy.get_rostime().to_sec()
        end_wait_time = 0.0
        epsilon = 0.1
        v_plus = velocity + epsilon
        v_minus = velocity - epsilon
        while not rospy.is_shutdown():
            joint_data = self._check_joint_states_ready()
            roll_vel = joint_data.velocity[0]
            rospy.logdebug("VEL=" + str(roll_vel) + ", ?RANGE=[" + str(v_minus) + ","+str(v_plus)+"]")
            are_close = (roll_vel <= v_plus) and (roll_vel > v_minus)
            if are_close:
                rospy.logdebug("Reached Velocity!")
                end_wait_time = rospy.get_rostime().to_sec()
                break
            rospy.logdebug("Not there yet, keep waiting...")
            rate.sleep()
        delta_time = end_wait_time- start_wait_time
        rospy.logdebug("[Wait Time=" + str(delta_time)+"]")
        return delta_time
        

    def get_joints(self):
        return self.joints
    
    def get_odom(self):
        return self.odom

<p style="background:#407EAF;color:white;">**END my_cube_single_disk_env.py**</p>

## NEXT STEP: How to create the One Disk Moving Cube Learning env for walking on the Y-axis.